In [16]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("movie_analysis_with_names").master("local[1]").getOrCreate()

env: PYSPARK_PYTHON=/usr/local/bin/python3
env: PYSPARK_DRIVER_PYTHON=/usr/local/bin/python3
env: PYSPARK_DRIVER_PYTHON_OPTS='notebook'


In [17]:
import codecs

def loadMovieNames():
    movieNames = {}
    with codecs.open("./ml-100k/u.item",encoding = "ISO-8859-1", errors = "ignore") as f:
        for line in f:
            fields = line.split('|')
            movieNames[int(fields[0])] = fields[1]
    return movieNames

nameDict = spark.sparkContext.broadcast(loadMovieNames())


In [18]:
from pyspark.sql.types import IntegerType, LongType, StructField, StructType

schema = StructType([
    StructField("user_id", IntegerType(), True),
    StructField("movie_id", IntegerType(), True),
    StructField("rating", IntegerType(), True),
    StructField("timestamp", LongType(), True)
])

movies_data = spark.read.csv("./ml-100k/u.data", sep="\t", schema= schema)

In [19]:
from pyspark.sql.functions import count, udf, desc

movie_counts= movies_data.groupBy("movie_id").agg(count("movie_id").alias("count"))
movie_counts.show()

+--------+-----+
|movie_id|count|
+--------+-----+
|     496|  231|
|     471|  221|
|     463|   71|
|     148|  128|
|    1342|    2|
|     833|   49|
|    1088|   13|
|    1591|    6|
|    1238|    8|
|    1580|    1|
|    1645|    1|
|     392|   68|
|     623|   39|
|     540|   43|
|     858|    3|
|     737|   59|
|     243|  132|
|    1025|   44|
|    1084|   21|
|    1127|   11|
+--------+-----+
only showing top 20 rows



In [20]:
def lookupName(movieId):
    return nameDict.value[movieId]

lookupNameUDF = udf(lookupName)

In [21]:
movie_with_names = movie_counts.withColumn("Title", lookupNameUDF(movie_counts["movie_id"])).orderBy(desc("count"))

In [22]:
movie_with_names.show()

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/Library/Python/3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 477, in main
    ("%d.%d" % sys.version_info[:2], version))
Exception: Python in worker has different version 2.7 than that in driver 3.8, PySpark cannot run with different minor versions. Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.
